In [5]:
import pickle
from tqdm import tqdm
import networkx as nx
import pandas as pd
import numpy as np
import os
import codecs
from scipy.sparse import coo_matrix
import networkx as nx
import torch
import dgl

In [6]:
import psutil
import os
info = psutil.virtual_memory()
print(u'电脑总内存：%.4f GB' % (info.total / 1024 / 1024 / 1024) )
print(u'当前使用的总内存占比：',info.percent)
print(u'cpu个数：',psutil.cpu_count())
print(u'当前进程的内存使用：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024))

电脑总内存：7.9150 GB
当前使用的总内存占比： 89.8
cpu个数： 4
当前进程的内存使用：0.3290 GB


In [4]:
import pandas as pd
# train = pd.read_csv("../copy-malscan/split_train.csv")
train_path = "test_data.pkl"
df = open(train_path, "rb")
all_train_dict = pickle.load(df)
df.close()

ModuleNotFoundError: No module named 'scipy.sparse._coo'

In [17]:
with open('../APIGraph-master/src/res/method_entity_embedding_TransE.pkl', 'rb') as f:
    entity_embedding = pickle.load(f)

In [7]:
adj_all = all_train_dict["noedge_weight"]
all_sha = all_train_dict["all_sha"]
node_all = all_train_dict["node_all"]
all_sen_idx = all_train_dict["all_sen_idx"]
all_label = all_train_dict["all_label"]
cons_fea = all_train_dict["cons_fea"]
cons = all_train_dict["cons"]

In [10]:
import sys
sys.path.append("/home2/ykli/workspace/flow_graph_exp/")
from Models import GCN_OUR
model_dir = "../flow_graph_exp/GCN_OUR_CON_NEW.pth"
model = GCN_OUR(10,64,1)
model.load_state_dict(torch.load(model_dir)["model"])
print('loaded model from: ', model_dir)
model.eval()

loaded model from:  ../flow_graph_exp/GCN_OUR_CON_NEW.pth


GCN_OUR(
  (conv1): GraphConv(in=10, out=64, normalization=none, activation=None)
  (conv2): GraphConv(in=64, out=64, normalization=none, activation=None)
  (conv3): GraphConv(in=64, out=64, normalization=none, activation=None)
  (classify): Linear(in_features=192, out_features=1, bias=True)
  (attn_fc): Linear(in_features=128, out_features=1, bias=False)
  (pooling): MaxPooling()
)

In [11]:
commufile = "../flow_graph_exp/all_commu_new.pkl"
with open(commufile, 'rb') as f:
    commu = pickle.load(f)

In [12]:
commu = np.array(commu)[train[train.test==True].index.values]

/tmp/ipykernel_1960895/1570822989.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  commu = np.array(commu)[train[train.test==True].index.values]


In [13]:
test_df = train[train.test!=False]
with open("../flow_graph_exp/edge_weight_new_con.pkl", 'rb') as f:
    final_edge = pickle.load(f)
all_edge_weight = final_edge["all_edge_weight"]
all_sens_edge_weight = final_edge["all_sens_edge_weight"]
all_edge_weight = np.array(all_edge_weight)
all_ww = all_edge_weight[test_df.index.values]

/tmp/ipykernel_1960895/136646228.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_edge_weight = np.array(all_edge_weight)


In [18]:
from itertools import product
import copy
from tqdm import tqdm

correct_indices = []
# all_graphs = X
# all_labels = Y
all_g = []
all_pred = []

all_C = []
all_S = []
all_N = []
all_sens = []
all_L = []

model.cuda()

for i in tqdm(range(len(all_label))):
    sample = adj_all[i].T
    g = dgl.from_scipy(sample)
    num_nodes = sample.shape[0]
    nodes = node_all[i]
    cc = cons_fea[i]
    c = cons[i]
    fea = np.zeros((num_nodes, 10))
    for ks in cc:
        if ks>=num_nodes:
            continue
        temp_node = nodes[ks][1:-1].split(":")
        ids = temp_node[0] + "." + temp_node[1].strip().split(" ")[-1].split("(")[0]
        if ids in entity_embedding:
            fea[ks] = entity_embedding[ids]
    
    edge_weight  = all_ww[i]
    g.edata['h'] = torch.tensor(edge_weight).float()
    
    with torch.no_grad():
        # for baseline
        # fea = g.in_degrees().view(-1, 1).float()
        # g.ndata['h'] = fea
        g.ndata['h'] = torch.tensor(fea).float()
        # g = g.to("cuda")
        all_g.append(copy.deepcopy(g))
        preds = model.cpu()(g).detach()
        
    all_pred.append(preds)
    all_L.append(all_label[i])
    all_C.append(c)
    all_S.append(cc)
    all_N.append(nodes)

100%|██████████| 4004/4004 [00:34<00:00, 115.68it/s]


In [19]:
all_pred = torch.cat(all_pred).reshape(-1).sigmoid()
isright = np.array((all_pred>0.5).cpu())==np.array(all_L)
selected_indices = np.where(isright==True)[0]
correct_indices = selected_indices

In [22]:
pred = (all_pred.reshape(-1)>0.5).cpu().detach().numpy().astype(np.int32)

In [24]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
def get_metric(test_Y, y_pred):
    f1 = f1_score(y_true=test_Y, y_pred=y_pred)
    precision = precision_score(y_true=test_Y, y_pred=y_pred)
    recall = recall_score(y_true=test_Y, y_pred=y_pred)
    accuracy = accuracy_score(y_true=test_Y, y_pred=y_pred)
    TP = np.sum(np.multiply(test_Y, y_pred))
    FP = np.sum(np.logical_and(np.equal(test_Y, 0), np.equal(y_pred, 1)))
    FN = np.sum(np.logical_and(np.equal(test_Y, 1), np.equal(y_pred, 0)))
    TN = np.sum(np.logical_and(np.equal(test_Y, 0), np.equal(y_pred, 0)))

    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    TNR = TN / (TN + FP)
    FNR = FN / (TP + FN)
    return f1, precision, recall, accuracy, TPR, FPR, TNR, FNR

In [25]:
get_metric(all_L, pred)

(0.9592508225765629,
 0.9722934838378656,
 0.9465534465534465,
 0.9597902097902098,
 0.9465534465534465,
 0.026973026973026972,
 0.973026973026973,
 0.05344655344655345)